In [18]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
libffm = pd.read_csv('DataSet\lib_dataset\lib_data.csv')

In [20]:
libffm

,target,user_id,problem_number,user_tier,problem_tier,problem_algorithm,cat_problem_avg_try,user_weak_algorithm
0,0,27612,3189,21,17,[26 52],4,[121 16 87 26 18 17 126 82 117 22]
1,0,27612,6901,21,11,[100 162 47],1,[121 16 87 26 18 17 126 82 117 22]
2,1,27612,6900,21,7,[ 18 23 126],2,[121 16 87 26 18 17 126 82 117 22]
3,1,27612,3443,21,18,[121 136 171 146],4,[121 16 87 26 18 17 126 82 117 22]
4,3,27612,6844,21,9,[121 114 20 163 87 127],3,[121 16 87 26 18 17 126 82 117 22]
...,...,...,...,...,...,...,...,...
2604305,0,30805,354,15,6,[126 47],2,[ 16 17 121 22 26 126 117 87 15 69]
2604306,0,30805,4969,15,8,[18],2,[ 16 17 121 22 26 126 117 87 15 69]
2604307,0,30805,4967,15,7,[18 87],3,[ 16 17 121 22 26 126 117 87 15 69]
2604308,0,30805,4968,15,11,[22 16 17 18 87],4,[ 16 17 121 22 26 126 117 87 15 69]


In [21]:
import re
import ast

def process_string(input_str):
    numbers = re.findall(r'\d+', input_str)
    return [int(num) for num in numbers]

# 데이터프레임의 열에 대해 위 함수를 적용하여 값 변환
libffm['problem_algorithm'] = libffm['problem_algorithm'].apply(process_string)
libffm['user_weak_algorithm'] = libffm['user_weak_algorithm'].apply(process_string)

X = libffm.iloc[:,1:]
y = libffm.iloc[:,:1]

In [22]:
X

,user_id,problem_number,user_tier,problem_tier,problem_algorithm,cat_problem_avg_try,user_weak_algorithm
0,27612,3189,21,17,"[26, 52]",4,"[121, 16, 87, 26, 18, 17, 126, 82, 117, 22]"
1,27612,6901,21,11,"[100, 162, 47]",1,"[121, 16, 87, 26, 18, 17, 126, 82, 117, 22]"
2,27612,6900,21,7,"[18, 23, 126]",2,"[121, 16, 87, 26, 18, 17, 126, 82, 117, 22]"
3,27612,3443,21,18,"[121, 136, 171, 146]",4,"[121, 16, 87, 26, 18, 17, 126, 82, 117, 22]"
4,27612,6844,21,9,"[121, 114, 20, 163, 87, 127]",3,"[121, 16, 87, 26, 18, 17, 126, 82, 117, 22]"
...,...,...,...,...,...,...,...
2604305,30805,354,15,6,"[126, 47]",2,"[16, 17, 121, 22, 26, 126, 117, 87, 15, 69]"
2604306,30805,4969,15,8,[18],2,"[16, 17, 121, 22, 26, 126, 117, 87, 15, 69]"
2604307,30805,4967,15,7,"[18, 87]",3,"[16, 17, 121, 22, 26, 126, 117, 87, 15, 69]"
2604308,30805,4968,15,11,"[22, 16, 17, 18, 87]",4,"[16, 17, 121, 22, 26, 126, 117, 87, 15, 69]"


In [23]:
y

,target
0,0
1,0
2,1
3,1
4,3
...,...
2604305,0
2604306,0
2604307,0
2604308,0


numpy.ndarray

In [27]:
from sklearn.preprocessing import MultiLabelBinarizer


# MultiLabelBinarizer를 사용하여 전체 리스트 원핫 인코딩
mlb_pa = MultiLabelBinarizer()
onehot_encoded_pa = pd.DataFrame(mlb_pa.fit_transform(X['problem_algorithm']), columns=[f"{col}_pa" for col in mlb_pa.classes_])

mlb_ua = MultiLabelBinarizer()
onehot_encoded_ua = pd.DataFrame(mlb_ua.fit_transform(X['user_weak_algorithm']), columns=[f"{col}_ua" for col in mlb_ua.classes_])

# 인코딩 결과를 원본 데이터프레임에 추가
X = pd.concat([X, onehot_encoded_pa], axis=1)
X.drop(['problem_algorithm'], axis=1, inplace=True)

X = pd.concat([X, onehot_encoded_ua], axis=1)
X.drop(['user_weak_algorithm'], axis=1, inplace=True)
# 결과 출력
X

,user_id,problem_number,user_tier,problem_tier,cat_problem_avg_try,0_pa,1_pa,2_pa,3_pa,4_pa,...,146_ua,150_ua,151_ua,152_ua,154_ua,158_ua,162_ua,163_ua,164_ua,167_ua
0,27612,3189,21,17,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,27612,6901,21,11,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27612,6900,21,7,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,27612,3443,21,18,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,27612,6844,21,9,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604305,30805,354,15,6,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2604306,30805,4969,15,8,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2604307,30805,4967,15,7,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2604308,30805,4968,15,11,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
y

,target
0,0
1,0
2,1
3,1
4,3
...,...
2604305,0
2604306,0
2604307,0
2604308,0


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

model = CatBoostRegressor(iterations=300, learning_rate=0.001, 
        depth=8, 
        #cat_features=list(X.columns),
        cat_features=['user_id', 'problem_number', 'user_tier', 'problem_tier', 'cat_problem_avg_try'],
        loss_function = 'RMSE')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'rmse: {rmse}')

0:	learn: 3.0943287	total: 1.59s	remaining: 7m 55s
1:	learn: 3.0939015	total: 3.1s	remaining: 7m 42s
2:	learn: 3.0934754	total: 4.54s	remaining: 7m 29s
3:	learn: 3.0930520	total: 5.2s	remaining: 6m 24s
4:	learn: 3.0926281	total: 5.91s	remaining: 5m 48s
5:	learn: 3.0922026	total: 6.61s	remaining: 5m 24s
6:	learn: 3.0917763	total: 7.29s	remaining: 5m 5s
7:	learn: 3.0913522	total: 8.04s	remaining: 4m 53s
8:	learn: 3.0909323	total: 9.12s	remaining: 4m 54s
9:	learn: 3.0905114	total: 10s	remaining: 4m 50s
10:	learn: 3.0900900	total: 10.7s	remaining: 4m 41s
11:	learn: 3.0896696	total: 11.5s	remaining: 4m 35s
12:	learn: 3.0892507	total: 12.1s	remaining: 4m 28s
13:	learn: 3.0888311	total: 12.9s	remaining: 4m 22s
14:	learn: 3.0884121	total: 13.6s	remaining: 4m 18s
15:	learn: 3.0879938	total: 14.5s	remaining: 4m 17s
16:	learn: 3.0875800	total: 15.5s	remaining: 4m 17s
17:	learn: 3.0871637	total: 16.4s	remaining: 4m 16s
18:	learn: 3.0867481	total: 17.4s	remaining: 4m 17s
19:	learn: 3.0863332	total:

KeyboardInterrupt: 

In [32]:
pd.DataFrame(y_pred)

,0
0,1.542685
1,2.307447
2,1.351590
3,1.403492
4,1.777105
...,...
194485,1.506509
194486,1.693104
194487,1.364044
194488,1.417860


In [36]:
y_test

,target_field
491428,0
454184,0
716131,0
869182,0
892784,1
...,...
316178,1
930254,0
355842,0
214253,0


In [34]:
y['target'] = y['target'].apply(lambda x: 1 if x>0 else 0)

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

model = CatBoostClassifier(iterations=1000, learning_rate=0.001, 
        depth=10, cat_features=list(X.columns),
        loss_function = 'Logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy =  accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred)

print(f'accuracy: {accuracy}, auc_score = {auc_score}')

0:	learn: 0.6929728	total: 2.36s	remaining: 39m 21s
1:	learn: 0.6927989	total: 4.89s	remaining: 40m 41s
2:	learn: 0.6926251	total: 6.6s	remaining: 36m 34s
3:	learn: 0.6924523	total: 8.62s	remaining: 35m 45s
4:	learn: 0.6922783	total: 10.7s	remaining: 35m 22s
5:	learn: 0.6921057	total: 12.2s	remaining: 33m 48s
6:	learn: 0.6919336	total: 14.1s	remaining: 33m 18s
7:	learn: 0.6917618	total: 16.4s	remaining: 33m 54s
8:	learn: 0.6915892	total: 18.3s	remaining: 33m 39s
9:	learn: 0.6914174	total: 20.1s	remaining: 33m 6s
10:	learn: 0.6912462	total: 22s	remaining: 33m 1s
11:	learn: 0.6910785	total: 23.8s	remaining: 32m 37s
12:	learn: 0.6909088	total: 25.5s	remaining: 32m 19s
13:	learn: 0.6907397	total: 27.4s	remaining: 32m 8s
14:	learn: 0.6905701	total: 29.4s	remaining: 32m 11s
15:	learn: 0.6904021	total: 31s	remaining: 31m 47s
16:	learn: 0.6902335	total: 32.7s	remaining: 31m 30s
17:	learn: 0.6900646	total: 34.6s	remaining: 31m 30s
18:	learn: 0.6898979	total: 36.5s	remaining: 31m 23s
19:	learn: 

In [40]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

model = XGBClassifier()
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

accuracy =  accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred)

print(f'accuracy: {accuracy}, auc_score = {auc_score}')

accuracy: 0.6528613296313435, auc_score = 0.6529281257464702
